**1\. Deploy a Spark cluster (GCP Dataproc)**

**2\. Store data in HDFS**

*   Load the data from Kaggle ("https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce")
*   Use #!/bin/bash "curl -L -o ~/olist/brazilian-ecommerce.zip https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce"

*   Unizp using "unizp /brazilian-ecommerce.zip -d /data/"

**3\. Use Spark to interact with data.**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("OlistData") \
.getOrCreate()

25/09/15 05:15:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [4]:
!hadoop fs -ls /data/olist/

Found 9 items
-rw-r--r--   2 nileshnandan_ts hadoop    9033957 2025-09-15 05:11 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop   61273883 2025-09-15 05:11 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop   15438671 2025-09-15 05:11 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop    5777138 2025-09-15 05:11 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop   14451670 2025-09-15 05:11 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop   17654914 2025-09-15 05:11 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop    2379446 2025-09-15 05:11 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop     174703 2025-09-15 05:11 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 nileshnandan_ts hadoop       2613 2025-09-15 05:11 /data/olist/product_category_name_translation.csv


In [6]:
hdfs_path = "/data/olist/"

In [7]:
customers_df = spark.read.csv(hdfs_path+"olist_customers_dataset.csv", header=True, inferSchema=True)
geo_df = spark.read.csv(hdfs_path+"olist_geolocation_dataset.csv", header=True, inferSchema=True)
items_df = spark.read.csv(hdfs_path+"olist_order_items_dataset.csv", header=True, inferSchema=True)
payments_df = spark.read.csv(hdfs_path+"olist_order_payments_dataset.csv", header=True, inferSchema=True)
reviews_df = spark.read.csv(hdfs_path+"olist_order_reviews_dataset.csv", header=True, inferSchema=True)
orders_df = spark.read.csv(hdfs_path+"olist_orders_dataset.csv", header=True, inferSchema=True)
products_df = spark.read.csv(hdfs_path+"olist_products_dataset.csv", header=True, inferSchema=True)
sellers_df = spark.read.csv(hdfs_path+"olist_sellers_dataset.csv", header=True, inferSchema=True)
category_name_df = spark.read.csv(hdfs_path+"product_category_name_translation.csv", header=True, inferSchema=True)

In [10]:
#data leakage check

print(f"Total rows of customers table: {customers_df.count()}")
print(f"Total rows of orders table: {orders_df.count()}")

Total rows of customers table: 99441
Total rows of orders table: 99441


In [16]:
#Null and duplicates

from pyspark.sql.functions import count, when, col

customers_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customers_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [17]:
customers_df.groupby('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [19]:
#Customer State distribution
customers_df.groupBy('customer_state').count().orderBy('count', ascending=False).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [21]:
orders_df.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [23]:
#Order status distribution
orders_df.groupBy('order_status').count().orderBy('count', ascending=False).show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [24]:
payments_df.show(5)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [25]:
payments_df.groupBy('payment_type').count().orderBy('count', ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [26]:
items_df.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [28]:
#Top selling products
from pyspark.sql.functions import sum
items_df.groupBy('product_id').agg(sum('price').alias('Total_sales')).orderBy('Total_sales', ascending=False).show()

+--------------------+------------------+
|          product_id|       Total_sales|
+--------------------+------------------+
|bb50f2e236e5eea01...|           63885.0|
|6cdd53843498f9289...| 54730.20000000005|
|d6160fb7873f18409...|48899.340000000004|
|d1c427060a0f73f6b...| 47214.51000000006|
|99a4788cb24856965...|43025.560000000085|
|3dd2a17168ec895c7...| 41082.60000000005|
|25c38557cf793876c...| 38907.32000000001|
|5f504b3a1c75b73d6...|37733.899999999994|
|53b36df67ebb7c415...| 37683.42000000001|
|aca2eb7d00ea1a7b8...| 37608.90000000007|
|e0d64dcfaa3b6db5c...|          31786.82|
|d285360f29ac7fd97...|31623.809999999983|
|7a10781637204d8d1...|           30467.5|
|f1c7f353075ce59d8...|          29997.36|
|f819f0c84a64f02d3...|29024.479999999996|
|588531f8ec37e7d5f...|28291.989999999998|
|422879e10f4668299...|26577.219999999972|
|16c4e87b98a9370a9...|           25034.0|
|5a848e4ab52fd5445...|24229.029999999962|
|a62e25e09e05e6faf...|           24051.0|
+--------------------+------------

In [35]:
#Average delivery time
from pyspark.sql.functions import col, datediff, to_date

delivery_df = orders_df.select('order_id', 'order_purchase_timestamp', 'order_delivered_customer_date').withColumn('Date_diff', datediff(col('order_delivered_customer_date'), col('order_purchase_timestamp')))
delivery_df.show(5)

+--------------------+------------------------+-----------------------------+---------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|Date_diff|
+--------------------+------------------------+-----------------------------+---------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|        8|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|       14|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|        9|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|       14|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|        3|
+--------------------+------------------------+-----------------------------+---------+
only showing top 5 rows



In [37]:
delivery_df.orderBy('Date_diff', ascending=False).show(5)

+--------------------+------------------------+-----------------------------+---------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|Date_diff|
+--------------------+------------------------+-----------------------------+---------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|      210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|      208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|      196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|      195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|      195|
+--------------------+------------------------+-----------------------------+---------+
only showing top 5 rows



In [40]:
from pyspark.sql.functions import avg

delivery_df.select(avg('Date_diff').alias("Average delivery days")).show()

+---------------------+
|Average delivery days|
+---------------------+
|   12.497336125046644|
+---------------------+

